# [鐵達尼生存預測(Kaggle)](https://www.kaggle.com/c/titanic)
以下用鐵達尼生存預測資料, 觀察計數編碼與特徵雜湊的效果

In [35]:
# 做完特徵工程前的所有準備
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', "Survived"], axis = 1)
df_test = df_test.drop(['PassengerId'], axis = 1)
df = pd.concat([df_train, df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [36]:
# 找出object欄位
df = df.select_dtypes(include=['object'])
# object_features = []
# for dtype, feature in zip(df.dtypes, df.columns):
#     if dtype == 'object':
#         object_features.append(feature)
# print(f'{len(object_features)} object Features : {object_features}\n')
# df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


In [37]:
# 觀察欄位相異值數量
df.apply(pd.Series.nunique)

Name        1307
Sex            2
Ticket       929
Cabin        187
Embarked       4
dtype: int64

In [38]:
# 對照組: Labelencoding + LogisticRegression
df_le = pd.DataFrame()
for i in df.columns:
    df_le[i] = LabelEncoder().fit_transform(df[i])
train_X = df_le[:train_num]
# estimator = LogisticRegression()
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv = 5).mean())
# print(cross_val_score(estimator, train_X, train_Y, cv = 5).mean())
df_le.head()

0.780004837244799


,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


In [41]:
# 加上'Ticket'欄位的計數編碼
count_df = df.groupby('Ticket')['Name'].agg({'Ticket_Count':'size'}).reset_index()
df = pd.merge(df, count_df, on = ['Ticket'], how = 'left')
count_df.sort_values(by=['Ticket_Count'], ascending = False).head(10)

C:\Users\TL_Yu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


,Ticket,Ticket_Count
778,CA. 2343,11
104,1601,8
775,CA 2144,8
335,3101295,7
454,347077,7
459,347082,7
847,S.O.C. 14879,7
824,PC 17608,7
123,19950,6
49,113781,6


In [42]:
df.head()

,Name,Sex,Ticket,Cabin,Embarked,Ticket_Count
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S,1
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,2
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S,1
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,2
4,"Allen, Mr. William Henry",male,373450,None,S,1


In [43]:
df.select_dtypes(include=['object']).columns

Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')

In [45]:
df_le = pd.DataFrame()
for i in df.select_dtypes(include=['object']).columns:
    df_le[i] = LabelEncoder().fit_transform(df[i])
df_le['Ticket_Count'] = df['Ticket_Count']
train_X = df_le[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_le.head()

0.7811221556805532


,Name,Sex,Ticket,Cabin,Embarked,Ticket_Count
0,155,1,720,185,3,1
1,286,0,816,106,0,2
2,523,0,914,185,3,1
3,422,0,65,70,3,2
4,22,1,649,185,3,1


In [48]:
# 'Ticket'特徵雜湊 + 邏輯斯迴歸
df_le = pd.DataFrame()
for i in df.select_dtypes(include=['object']).columns:
    df_le[i] = LabelEncoder().fit_transform(df[i])
df_le['Ticket_Hash'] = df['Ticket'].map(lambda x:hash(x) % 10)
train_X = df_le[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_le.head()

0.7811221556805532


,Name,Sex,Ticket,Cabin,Embarked,Ticket_Hash
0,155,1,720,185,3,7
1,286,0,816,106,0,9
2,523,0,914,185,3,5
3,422,0,65,70,3,3
4,22,1,649,185,3,0


In [51]:
# 'Ticket'計數編碼 + 'Ticket'特徵雜湊 + 邏輯斯迴歸
df_le = pd.DataFrame()
for i in df.select_dtypes(include=['object']).columns:
    df_le[i] = LabelEncoder().fit_transform(df[i])
df_le['Ticket_Hash'] = df['Ticket'].map(lambda x:hash(x) % 10)
df_le['Ticket_Count'] = df['Ticket_Count']
train_X = df_le[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_le.head()

0.7788749646693172


,Name,Sex,Ticket,Cabin,Embarked,Ticket_Hash,Ticket_Count
0,155,1,720,185,3,7,1
1,286,0,816,106,0,9,2
2,523,0,914,185,3,5,1
3,422,0,65,70,3,3,2
4,22,1,649,185,3,0,1


# 作業1
* 參考範例，將鐵達尼的艙位代碼( 'Cabin' )欄位使用特徵雜湊 / 標籤編碼 / 計數編碼三種轉換後， 與其他數值型欄位一起預估生存機率

# 作業2
* 承上題，三者比較效果何者最好?

In [96]:
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'], axis = 1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train, df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [97]:
df_number = df.select_dtypes('number')
df_number = df_number.fillna(0)
df_Cabin = df.loc[:,['Cabin']]
df_Cabin = df_Cabin.fillna('None')
df2 = pd.concat([df_number, df_Cabin], axis=1)
train_num = train_Y.shape[0]
df2.head()

,Pclass,Age,SibSp,Parch,Fare,Cabin
0,3,22.0,1,0,7.2500,None
1,1,38.0,1,0,71.2833,C85
2,3,26.0,0,0,7.9250,None
3,1,35.0,1,0,53.1000,C123
4,3,35.0,0,0,8.0500,None


In [98]:
# 標籤編碼
df2['Cabin_le'] = LabelEncoder().fit_transform(df2['Cabin'])
df2.head()

,Pclass,Age,SibSp,Parch,Fare,Cabin,Cabin_le
0,3,22.0,1,0,7.2500,None,185
1,1,38.0,1,0,71.2833,C85,106
2,3,26.0,0,0,7.9250,None,185
3,1,35.0,1,0,53.1000,C123,70
4,3,35.0,0,0,8.0500,None,185


In [99]:
# 計數編碼
count_df = df2.groupby(['Cabin'])['Cabin'].agg({'Cabin_Count':'size'}).reset_index()
df2 = df2.merge(count_df, how='left', on = 'Cabin')

C:\Users\TL_Yu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [101]:
# feature Hashing
df2['Cabin_Hash'] = df2['Cabin'].map(lambda x:hash(x) % 10)
df2.head(10)

,Pclass,Age,SibSp,Parch,Fare,Cabin,Cabin_le,Cabin_Count,Cabin_Hash
0,3,22.0,1,0,7.2500,None,185,1014,3
1,1,38.0,1,0,71.2833,C85,106,2,2
2,3,26.0,0,0,7.9250,None,185,1014,3
3,1,35.0,1,0,53.1000,C123,70,2,6
4,3,35.0,0,0,8.0500,None,185,1014,3
5,3,0.0,0,0,8.4583,None,185,1014,3
6,1,54.0,0,0,51.8625,E46,163,2,3
7,3,2.0,3,1,21.0750,None,185,1014,3
8,3,27.0,0,2,11.1333,None,185,1014,3
9,2,14.0,1,0,30.0708,None,185,1014,3


In [105]:
# Label + Logistic
df_le = df2.drop(['Cabin','Cabin_Count','Cabin_Hash'],axis=1)
train_X = df_le[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv = 5).mean())

0.6949063173920583


In [106]:
# count + Logistic
df_count = df2.drop(['Cabin','Cabin_le','Cabin_Hash'],axis=1)
train_X = df_count[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv = 5).mean())

0.6937322925455007


In [108]:
# hash + Logistic
df_ha = df2.drop(['Cabin','Cabin_le','Cabin_Count'],axis=1)
train_X = df_ha[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv = 5).mean())

0.7038384459535418


In [110]:
# Label + Gradient
df_le = df2.drop(['Cabin','Cabin_Count','Cabin_Hash'],axis=1)
train_X = df_le[:train_num]
estimator = GradientBoostingClassifier()
print(cross_val_score(estimator, train_X, train_Y, cv = 5).mean())

0.7172774397498818


In [111]:
# count + Gradient
df_count = df2.drop(['Cabin','Cabin_le','Cabin_Hash'],axis=1)
train_X = df_count[:train_num]
estimator = GradientBoostingClassifier()
print(cross_val_score(estimator, train_X, train_Y, cv = 5).mean())

0.7329258912109319


In [112]:
# hash + Gradient
df_ha = df2.drop(['Cabin','Cabin_le','Cabin_Count'],axis=1)
train_X = df_ha[:train_num]
estimator = GradientBoostingClassifier()
print(cross_val_score(estimator, train_X, train_Y, cv = 5).mean())

0.7262601395069911
